This notebook shows a few exploratory examples of decoding factors from call option prices simulated from a Heston-SLV model.

In [1]:
import os
import sys
import pickle
import random
import warnings
warnings.filterwarnings("ignore")

# add the path of packages to system path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# add packages for computation
import numpy as np
import pandas as pd
import marketmodel.loader as loader
import marketmodel.utils as utils

from marketmodel.factors import PrepTrainData, DecodeFactor
from marketmodel.loader import DataHestonSlv
from marketmodel.neuralsde import Train, Simulate, Model

# load configurations
hp_model_S = utils.Config.hp_model_S
hp_model_mu = utils.Config.hp_model_mu
hp_model_xi = utils.Config.hp_model_xi
hp_sde_transform = utils.Config.hp_sde_transform

# set paths
hestonslv_fname = '../input/sim_hestonslv.pkl'
checkpoint_model_S_initial = '../output/checkpoint/initial_model_S/model_S_0'

# Load and pre-process data

In [2]:
# load Heston-SLV simulation data
St, vt, list_exp, list_mny, cs_ts_raw, cs_ts, mask_quality_value, \
Ts, ks, mat_A, vec_b = loader.load_hestonslv_data(hestonslv_fname)

# load the trained initial model for S
X_S, Y_S = PrepTrainData.prep_data_model_S_initial(
    St, cs_ts, max_PC=7, factor_multiplier=1e5)

model_S_initial = Model.construct_S(X_S.shape[1], X_S.shape[0], 
                                    hp_model_S['pruning_sparsity'],
                                    hp_model_S['validation_split'],
                                    hp_model_S['batch_size'],
                                    hp_model_S['epochs'])
model_S_initial.load_weights(checkpoint_model_S_initial)

# calculate derivatives for the normalised call prices
cT_ts, cm_ts, cmm_ts = PrepTrainData.calc_call_derivatives(
    list_mny, list_exp, cs_ts_raw, mask_quality_value)

z_ts = PrepTrainData.calc_zt(cT_ts, cm_ts, cmm_ts, model_S_initial, X_S)

# define the effective constraints through their inequality constraints
mask_redundant = DecodeFactor.find_redundant_constraints(-mat_A, -vec_b)
mat_A_nec = mat_A[~mask_redundant, :]
vec_b_nec = vec_b[~mask_redundant]

2022-06-17 16:59:16.714811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████████████████████████████████████████████| 10000/10000 [00:42<00:00, 236.19it/s]


313/313 [==============================] - 0s 889us/step


100%|██████████████████████████████████████████████████████| 281/281 [00:00<00:00, 608.49it/s]


# Decode factors

In [3]:
# calculate the constant term G0
G0 = cs_ts.mean(axis=0)
res0 = cs_ts - G0[None, :]

## One dynamic arbitrage factor

In [4]:
# parameters
n_da_factor = 1  # number of dynamic arbitrage factor

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

Gx = G_da
X = xi_da

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

313/313 [==============================] - 0s 835us/step
Residual norm: 0.3072
MAPE:          24.37%
PSAS:          60.67%
PDA:           3.51%


## Two dynamic arbitrage factors

In [5]:
# parameters
n_da_factor = 2  # number of dynamic arbitrage factor

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

Gx = G_da
X = xi_da

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

313/313 [==============================] - 0s 888us/step
Residual norm: 0.2852
MAPE:          17.22%
PSAS:          69.85%
PDA:           0.75%


## One dynamic arbitrage factor and one static arbitrage factor

In [6]:
# parameters
n_da_factor = 1  # number of dynamic arbitrage factor
n_sa_factor = 1
n_PC = 6

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

res1 = res0 - xi_da.dot(G_da)

# decode static arbitrage factors
rhs0 = vec_b_nec - G0.dot(mat_A_nec.T)
rhs1 = rhs0[:, None] - mat_A_nec.dot(G_da.reshape((-1, 1))). \
    dot(xi_da.reshape((1, -1)))

G_sa, xi_sa, weights = DecodeFactor.decode_stcarb_factor(
    res1, n_sa_factor, n_PC, mat_A_nec, rhs1)

313/313 [==============================] - 0s 911us/step
Searching for the static-arbitrage minimisation direction for projection.
****** Py-BOBYQA Results ******
Solution xmin = [ 0.96461803 -0.25353509 -0.53838957  0.05788529  0.04651391 -0.01545341]
Objective value f(xmin) = -9963.814951
Needed 4376 objective evaluations (at 4376 points)
Did a total of 32 runs
Approximate gradient = [ -27.2596052  -109.67368418  201.95248586 -123.34596494  -76.76461807
 -146.39056229]
Approximate Hessian = [[-2.20586069e+10 -1.07025971e+10 -1.74397814e+10 -4.70644908e+10
  -5.37598755e+09  2.17433290e+10]
 [-1.07025971e+10  2.48556348e+10 -6.98958036e+09  7.08156703e+10
  -1.14145649e+10 -4.88926199e+10]
 [-1.74397814e+10 -6.98958036e+09  4.13123070e+09 -4.93620968e+10
   3.22913894e+10  2.25795178e+10]
 [-4.70644908e+10  7.08156703e+10 -4.93620968e+10  1.13419578e+11
  -6.10612397e+09 -8.08627554e+09]
 [-5.37598755e+09 -1.14145649e+10  3.22913894e+10 -6.10612397e+09
   1.92034148e+10  3.47523213e+1

In [7]:
# combine factors
Gx = np.vstack((G_da, G_sa))
X = np.hstack((xi_da, xi_sa))

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.1850
MAPE:          3.85%
PSAS:          0.37%
PDA:           2.96%


## One dynamic factor and one statistical accuracy factor

In [8]:
# parameters
n_da_factor = 1  # number of dynamic arbitrage factor
n_pca_factor = 1

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

res1 = res0 - xi_da.dot(G_da)

G_pca, xi_pca = DecodeFactor.decode_pca_factor(res1, n_pca_factor)

# combine factors
Gx = np.vstack((G_da, G_pca))
X = np.hstack((xi_da, xi_pca))

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

313/313 [==============================] - 0s 1ms/step
Residual norm: 0.1129
MAPE:          5.11%
PSAS:          28.11%
PDA:           3.21%


## Two statistical accuracy factors

In [9]:
# parameters
n_pca_factor = 2

G_pca, xi_pca = DecodeFactor.decode_pca_factor(res0, n_pca_factor)

# combine factors
Gx = G_pca
X = xi_pca

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.0728
MAPE:          2.84%
PSAS:          39.44%
PDA:           4.46%
